In [4]:
from texttable import Texttable
from IPython.display import display, Latex
from math import gcd # Greatest common divisor
import matplotlib.pyplot as plt
import numpy as np

class Polynomial:
    def __init__(self, coefficients) -> None:
        self.coefficients = coefficients
        self.deg = len(coefficients) - 1

    def __add__(self, other) -> "Polynomial":
        if len(self.coefficients) > len(other.coefficients):
            result = self.coefficients.copy()
            for i in range(len(other.coefficients)):
                result[i] += other.coefficients[i]
        else:
            result = other.coefficients.copy()
            for i in range(len(self.coefficients)):
                result[i] += self.coefficients[i]
        return Polynomial(result)
    
    def __sub__(self, other) -> "Polynomial":
        if len(self.coefficients) >= len(other.coefficients):
            result = self.coefficients.copy()
            for i in range(len(other.coefficients)):
                result[i] -= other.coefficients[i]
        else:
            result = [-coefficient for coefficient in other.coefficients]
            for i in range(len(self.coefficients)):
                result[i] += self.coefficients[i]
        return Polynomial(result)
    
    def __mul__(self, other) -> "Polynomial":
        result = [0] * (len(self.coefficients) + len(other.coefficients) - 1)
        for i in range(len(self.coefficients)):
            for j in range(len(other.coefficients)):
                result[i + j] += self.coefficients[i] * other.coefficients[j]
        return Polynomial(result)

    def __truediv__(self, other: "Polynomial") -> list["Polynomial"]:
        temp_self = np.poly1d(self.coefficients)
        temp_other = np.poly1d(other.coefficients)
        quotient, remainder = np.polydiv(temp_self, temp_other)
        quotient = [num for num in quotient.c]
        remainder = [num for num in remainder.c]
        return [Polynomial(quotient), Polynomial(remainder)]

    def format_division(self, other) -> str:
        quotient, remainder = self.__truediv__(other)
        return [f"Q(x)={quotient}", f"R(x)={remainder}"]

    def __floordiv__(self, other) -> "Polynomial":
        return self.__truediv__(other)[0]

    def __str__(self) -> str:
        result = ""
        for i in range(len(self.coefficients)):
            power = len(self.coefficients) - i - 1
            if self.coefficients[i] == 0:
                continue
            elif self.coefficients[i] == 1 and power == 0:
                result += "+1"
            if self.coefficients[i] == -1 and power == 0:
                result += "-1"
            elif self.coefficients[i] == 1 and power == 1:
                result += "+x"
            elif self.coefficients[i] == -1 and power == 1:
                result += "-x"
            elif self.coefficients[i] == 1:
                result += f"+x^{power}"
            elif self.coefficients[i] == -1:
                result += f"-x^{power}"
            elif power == 0 and self.coefficients[i] > 0:
                result += f"+{self.coefficients[i]}"
            elif power == 0 and self.coefficients[i] < 0:
                result += f"{self.coefficients[i]}"
            elif power == 1 and self.coefficients[i] > 0:
                result += f"+{self.coefficients[i]}x"
            elif power == 1 and self.coefficients[i] < 0:
                result += f"{self.coefficients[i]}x"
            elif self.coefficients[i] > 0:
                result += f"+{self.coefficients[i]}x^{power}"
            elif self.coefficients[i] < 0:
                result += f"{self.coefficients[i]}x^{power}"
        if result == "":
            result = "0"
        if result[0] == "+":
            result = result[1:]
        return result


    def __repr__(self) -> str:
        return f"Polynomial(coefficients={self.coefficients})"
    
    def derivative(self) -> "Polynomial":
        result = []
        for i in range(len(self.coefficients)-1):
            power = len(self.coefficients) - i - 1
            result.append(self.coefficients[i] * power)
        return Polynomial(result)
    
    def all_derivatives(self) -> list["Polynomial"]:
        coeffs = self.coefficients.copy()
        result = []
        while len(coeffs) > 1:
            result.append(Polynomial(coeffs))
            coeffs = Polynomial(coeffs).derivative().coefficients
        return result
    
    def print_all_derivatives(self) -> None:
        for i in range(len(self.all_derivatives())):
            if i == 0:
                format_print(color="blue", text=f"Polynomial: ")
            else:
                format_print(color="blue", text=f"Derivative {i}")
            latex_print("f" + "'" * i + "(x)=" + str(self.all_derivatives()[i]))


def latex_print(object):
    return display(Latex(f"${object}$"))

def format_print(text, color, sep=" ", end="\n"):
    colors = {
        "purple": '\033[95m',
        "cyan": '\033[96m',
        "dark_cyan": '\033[36m',
        "blue": '\033[94m',
        "green": '\033[92m',
        "yellow": '\033[93m',
        "red": '\033[91m',
        "bold": '\033[1m',
        "underline": '\033[4m',
        "end": '\033[0m'
    }
    print(colors[color] + text + colors["end"], sep=sep, end=end)
